In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import boto3
import rasterio
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
import os

In [3]:
df = pd.read_csv("Data/DroneVF_CalVal/DroneDataList.csv")
df.head(23)

,Date,Location Name,Nearest Town,Scan Start Time (UTC),Scan End Time (UTC)
0,3/21/2022,Fairland March,"Fairland, MD",2022-03-21T15:33Z,2022-03-21T15:41Z
1,6/4/2022,White 2,"White, SD",2022-06-04T15:13Z,2022-06-04T15:21Z
2,6/4/2022,Oak Lake,"Hendricks, SD",2022-06-04T16:13Z,2022-06-04T16:26Z
3,6/5/2022,Lisbon,"Englevale, ND",2022-06-05T16:30Z,2022-06-05T16:41Z
4,7/22/2022,Glen Ullin,"Glen Ullin, ND",2022-07-22T19:44Z,2022-07-22T19:51Z
5,7/25/2022,Velva,"Velva, ND",2022-07-25T17:12Z,2022-07-25T17:17Z
6,7/26/2022,Tioga,"Tioga, ND",2022-07-26T16:01Z,2022-07-26T16:05Z
7,8/1/2022,Island Park,"Ashton, ID",2022-08-01T17:22Z,2022-08-01T17:27Z
8,9/16/2022,Fairland Sept,"Fairland, MD",2022-09-16T19:22Z,2022-09-16T19:27Z
9,10/6/2022,Taylors Island,"Taylors Island, MD",2022-10-06T19:48Z,2022-10-06T19:55Z


In [4]:
imgList = []
for file in os.listdir("Data/DroneVF_CalVal/"):
    if file.endswith('.tif'):
        imgList.append(file)

In [5]:
geometry = []
img_file = []
for row in df.iterrows():
    location = row[1]['Location Name']
    date = row[1]['Date'].split('/')
    date = "".join([date[2],date[0].zfill(2),date[1].zfill(2)])
    #print("".join(location.split(' ')))
    #print(date)
    for img in imgList:
        if ("".join(location.split(' ')) in img) and (date in img):
            #print(img)
            img_file.append(img)
            with rasterio.open("Data/DroneVF_CalVal/"+img) as ds:
                bounds = ds.bounds
                crs_utm = ds.crs
            bounds_4326 = transform_bounds(crs_utm, CRS.from_epsg(4326), *bounds)
            classified_geometry = box(*bounds_4326)
            geometry.append(classified_geometry)

            #print(f'match found for {location} {date} {img}')
            

df['geometry'] = geometry
df['img_file'] = img_file
gdf = gpd.GeoDataFrame(df,geometry=geometry, crs= 4326)
gdf.head()

,Date,Location Name,Nearest Town,Scan Start Time (UTC),Scan End Time (UTC),geometry,img_file
0,3/21/2022,Fairland March,"Fairland, MD",2022-03-21T15:33Z,2022-03-21T15:41Z,"POLYGON ((-76.92651 39.07932, -76.92651 39.082...",20220321_FairlandMarch_VF.tif
1,6/4/2022,White 2,"White, SD",2022-06-04T15:13Z,2022-06-04T15:21Z,"POLYGON ((-96.66459 44.46127, -96.66459 44.465...",20220604_White2_VF.tif
2,6/4/2022,Oak Lake,"Hendricks, SD",2022-06-04T16:13Z,2022-06-04T16:26Z,"POLYGON ((-96.53132 44.50790, -96.53132 44.513...",20220604_OakLake_VF.tif
3,6/5/2022,Lisbon,"Englevale, ND",2022-06-05T16:30Z,2022-06-05T16:41Z,"POLYGON ((-97.94155 46.43370, -97.94155 46.437...",20220605_Lisbon_VF.tif
4,7/22/2022,Glen Ullin,"Glen Ullin, ND",2022-07-22T19:44Z,2022-07-22T19:51Z,"POLYGON ((-101.85755 46.70109, -101.85755 46.7...",20220722_GlenUllin_VF.tif


In [6]:
bucket_name = 'opera-calval-database-dist'
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.client('s3')
s3_resource = session.resource('s3')

In [12]:
bucket = []
keys = []
imgNames = []
for row in gdf.iterrows():
    file = row[1]['img_file']
    filePath = "Data/DroneVF_CalVal/"+file
    imgName = file.strip('_VF.tif')
    key = f"Data/Drone_Imagery/{imgName}/{file}"
    #print(key)
    response = s3.upload_file(filePath, bucket_name, key)
    bucket.append(bucket_name)
    keys.append(key)
    imgNames.append(imgName)

In [15]:
gdf['bucket'] = bucket
gdf['key_VFtif'] = keys
gdf['Image_Name'] = imgNames
gdf.drop('img_file',axis=1,inplace=True)
gdf.head(23)

,Date,Location Name,Nearest Town,Scan Start Time (UTC),Scan End Time (UTC),geometry,bucket,key_VFtif,Image_Name
0,3/21/2022,Fairland March,"Fairland, MD",2022-03-21T15:33Z,2022-03-21T15:41Z,"POLYGON ((-76.92651 39.07932, -76.92651 39.082...",opera-calval-database-dist,Data/Drone_Imagery/20220321_FairlandMarch/2022...,20220321_FairlandMarch
1,6/4/2022,White 2,"White, SD",2022-06-04T15:13Z,2022-06-04T15:21Z,"POLYGON ((-96.66459 44.46127, -96.66459 44.465...",opera-calval-database-dist,Data/Drone_Imagery/20220604_White2/20220604_Wh...,20220604_White2
2,6/4/2022,Oak Lake,"Hendricks, SD",2022-06-04T16:13Z,2022-06-04T16:26Z,"POLYGON ((-96.53132 44.50790, -96.53132 44.513...",opera-calval-database-dist,Data/Drone_Imagery/20220604_OakLake/20220604_O...,20220604_OakLake
3,6/5/2022,Lisbon,"Englevale, ND",2022-06-05T16:30Z,2022-06-05T16:41Z,"POLYGON ((-97.94155 46.43370, -97.94155 46.437...",opera-calval-database-dist,Data/Drone_Imagery/20220605_Lisbon/20220605_Li...,20220605_Lisbon
4,7/22/2022,Glen Ullin,"Glen Ullin, ND",2022-07-22T19:44Z,2022-07-22T19:51Z,"POLYGON ((-101.85755 46.70109, -101.85755 46.7...",opera-calval-database-dist,Data/Drone_Imagery/20220722_GlenUllin/20220722...,20220722_GlenUllin
5,7/25/2022,Velva,"Velva, ND",2022-07-25T17:12Z,2022-07-25T17:17Z,"POLYGON ((-100.95858 48.03479, -100.95858 48.0...",opera-calval-database-dist,Data/Drone_Imagery/20220725_Velva/20220725_Vel...,20220725_Velva
6,7/26/2022,Tioga,"Tioga, ND",2022-07-26T16:01Z,2022-07-26T16:05Z,"POLYGON ((-102.82642 48.27608, -102.82642 48.2...",opera-calval-database-dist,Data/Drone_Imagery/20220726_Tioga/20220726_Tio...,20220726_Tioga
7,8/1/2022,Island Park,"Ashton, ID",2022-08-01T17:22Z,2022-08-01T17:27Z,"POLYGON ((-111.44792 44.21822, -111.44792 44.2...",opera-calval-database-dist,Data/Drone_Imagery/20220801_IslandPark/2022080...,20220801_IslandPark
8,9/16/2022,Fairland Sept,"Fairland, MD",2022-09-16T19:22Z,2022-09-16T19:27Z,"POLYGON ((-76.92720 39.07929, -76.92720 39.083...",opera-calval-database-dist,Data/Drone_Imagery/20220916_FairlandSep/202209...,20220916_FairlandSep
9,10/6/2022,Taylors Island,"Taylors Island, MD",2022-10-06T19:48Z,2022-10-06T19:55Z,"POLYGON ((-76.26020 38.48610, -76.26020 38.490...",opera-calval-database-dist,Data/Drone_Imagery/20221006_TaylorsIsland/2022...,20221006_TaylorsIsland


In [16]:
newRow_bytes = bytes(gdf.to_json(drop_id=True).encode('UTF-8'))
s3object = s3_resource.Object(bucket_name, 'drone_imagery.geojson')
s3object.put(Body=newRow_bytes)

{'ResponseMetadata': {'RequestId': '7808AEFQ1XG1AGCA',
  'HostId': 'nt7wCJGA4QokGpNqYOt/brTJQ88cQu3zVC2tsxuZqyvKAC5PYJOLYhLe5skwIYgZLWIiG9MbKMExMJTPdpkLoA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'nt7wCJGA4QokGpNqYOt/brTJQ88cQu3zVC2tsxuZqyvKAC5PYJOLYhLe5skwIYgZLWIiG9MbKMExMJTPdpkLoA==',
   'x-amz-request-id': '7808AEFQ1XG1AGCA',
   'date': 'Wed, 22 Mar 2023 18:39:15 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"48861fcdc063f706a95dc274b8f0d9bd"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"48861fcdc063f706a95dc274b8f0d9bd"',
 'ServerSideEncryption': 'AES256'}